<a href="https://colab.research.google.com/github/AV-Gorshkov/Python_netology/blob/main/PY_%D0%93%D0%B8%D0%B1%D1%80%D0%B8%D0%B4%D0%BD%D1%8B%D0%B5_%D1%80%D0%B5%D0%BA%D0%BE%D0%BC%D0%B5%D0%BD%D0%B4%D0%B0%D1%82%D0%B5%D0%BB%D1%8C%D0%BD%D1%8B%D0%B5_%D1%81%D0%B8%D1%81%D1%82%D0%B5%D0%BC%D1%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Задание

датасет MovieLens:

создание гибридной системы рекомендации

In [ ]:
# загрузка библиотеки surprise
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357291 sha256=64294533d721369718a7477d19041cf798488774f724df714b308ccdc14c557d
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [ ]:
from surprise import SVD
from surprise import KNNWithMeans, KNNBaseline,  KNNBasic
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split

from tqdm import tqdm_notebook

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import cityblock, cosine, euclidean, hamming, jaccard

import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
#  загрузка данных
param = r'/content/drive/My Drive/Нетология/ML/Рекомендательные системы/'

movies = pd.read_csv(f'{param}movies.csv')
ratings = pd.read_csv(f'{param}ratings.csv')

movies.head(5)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
# объединим таблицы с рейтингами и названиями фильмов

movies_ratings = movies.merge( ratings, on='movieId').reset_index(drop=True)
movies_ratings.dropna(inplace=True)

movies_ratings.head(5)

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483


## Вариант 1

In [ ]:
# датасет для Surprise

dataset = pd.DataFrame( {
    'uid'   : movies_ratings['userId'],
    'iid'   : movies_ratings['title'],
    'rating' : movies_ratings['rating']
})

# данные для создания модели

reader = Reader(rating_scale=(dataset['rating'].min(), dataset['rating'].max()))
data = Dataset.load_from_df(dataset, reader)

In [ ]:
# модель KNNBaseline
# модель рекомендации на рейтингах фильма

trainset, testset = train_test_split(data, test_size=0.15, random_state=42)

model_KNN = KNNBaseline(k=40, sim_options={'name': 'pearson_baseline', 'user_based': False})

model_KNN.fit(trainset)

test_pred = model_KNN.test(testset)
accuracy.rmse(test_pred, verbose=True)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8461


0.8460980885360158

In [ ]:
#  модель SVD

# trainset, testset = train_test_split(data, test_size=0.2, random_state=1)

# ml_svd = SVD(n_factors=20, n_epochs=20, random_state=1)
# ml_svd.fit(trainset)

# test_pred = ml_svd.test(testset)

# accuracy.rmse(test_pred, verbose=True)

# model SVD
#  рекомендации
# current_user_id = 2.0
# user_movies = movies_ratings[movies_ratings.userId == current_user_id].title.unique()

# scores = []
# titles = []

# for movie in movies_ratings.title.unique():
#     if movie in user_movies:
#         continue

#     scores.append(ml_svd.predict(uid=current_user_id, iid=movie).est)
#     titles.append(movie)

# best = np.argsort(scores)[-10:]

# for i in reversed(best):
#   print( titles[i], scores[i])

In [ ]:
# model_KNN
# рекомендации

current_user_id = 10.0
user_movies = movies_ratings[movies_ratings.userId == current_user_id].title.unique()

scores = []
titles = []

for movie in movies_ratings.title.unique():
    if movie in user_movies:
        continue

    scores.append(model_KNN.predict(uid=current_user_id, iid=movie).est)
    titles.append(movie)


sorted(scores)[-10:]

[4.284138477610336,
 4.358557940675762,
 4.374199084838914,
 4.378197085843423,
 4.444719880495544,
 4.539301551458315,
 4.585987287038421,
 4.661495038567163,
 4.744808374161465,
 4.9181700035577]

In [ ]:
#  модель рекомендации на жанрах фильма
#  NearestNeighbors подход (лгоритма ближайших соседей на жанрах)

# фун-ци парсинга жанров фильма

def change_string(g):
    return g.replace(' ', '').replace('-', '').replace('|', ' ')

movie_genres = [change_string(x) for x in movies.genres.values]

# Tfidf преобразование текста в вектор
count_vect = CountVectorizer()
x_train = count_vect.fit_transform(movie_genres)

tfidf_transformer = TfidfTransformer()
x_tfidf = tfidf_transformer.fit_transform(x_train)

model_neig = NearestNeighbors(n_neighbors=50, n_jobs=-1, metric='euclidean')
model_neig.fit(x_tfidf);

In [ ]:
title_genres = {}

for index, row in tqdm_notebook(movies.iterrows()):
    title_genres[row.title] = row.genres


<ipython-input-10-bd73bb19affb>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for index, row in tqdm_notebook(movies.iterrows()):


0it [00:00, ?it/s]

In [ ]:
# фун-ция рекомендации фильма

def recommend_for_user(user_id):

    # список фильмов для пользователя
    user_movies = movies_ratings [movies_ratings ["userId"] == user_id].sort_values('timestamp').title.values

    # последний просмотренный пользователем фильм
    last_user_movie = user_movies[-1]

    # жанры фильма преобразуем в векток TF-IDF
    movi_genres = change_string(title_genres[last_user_movie])

    #  получаем предсказание по жанру последнего фильма

    predict = count_vect.transform([movi_genres])
    x_tfidf2 = tfidf_transformer.transform(predict)

    res = model_neig.kneighbors(x_tfidf, return_distance=True)

    movies_to_score = movies.iloc[res[1][0]].title.values

    # для отобранных фильмов, предсказываем оценку на скрытых факторах
    scores = []
    title = []

    for movie in movies_to_score:
        if movie in user_movies:
            continue
        scores.append(model_KNN.predict(uid=user_id, iid=movie).est)
        title.append(movie)

    #  формируем датафрейм с рекомендациями
    best_indexes = np.argsort(scores)[-10:]
    mov = []
    scor = []

    for i in reversed(best_indexes):

        mov.append(title[i])
        scor.append(scores[i])

    return pd.DataFrame({"Movie": mov, "Rating": scor})

In [ ]:
#  рекомендации для пользователя
user_id = 17
print(f"Рекомендации для пользователя с ID = {user_id}.")
recommend_for_user(user_id)


Рекомендации для пользователя с ID = 17.


,Movie,Rating
0,Ponyo (Gake no ue no Ponyo) (2008),4.242064
1,"Monsters, Inc. (2001)",4.223143
2,Toy Story 2 (1999),4.217892
3,Kiki's Delivery Service (Majo no takkyûbin) (1...,4.171180
4,"SpongeBob SquarePants Movie, The (2004)",4.165480
5,Shrek (2001),4.143280
6,Watership Down (1978),4.121177
7,Inside Out (2015),4.118099
8,Moana (2016),4.098799
9,The Lego Movie (2014),4.033551


## Вариант 2

Модель lightfm

In [ ]:
!pip install lightfm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp310-cp310-linux_x86_64.whl size=808328 sha256=a4f3a7154ba25222056d628e012dd434965aff1e139db680a8283629dd254e8e
  Stored in directory: /root/.cache/pip/wheels/4f/9b/7e/0b256f2168511d8fa4dae4fae0200fdbd729eb424a912ad636
Successfully built lightfm


In [ ]:
from lightfm import LightFM
from lightfm.data import Dataset
from scipy import sparse

In [ ]:
movies_ratings.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483


In [ ]:
#  матрица взаимодействия

interactions = movies_ratings.groupby(['userId', 'title'])['rating'].count().unstack().fillna(0)

interactions.head()

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:

interactions_spaarse = sparse.csr_matrix(interactions)
interactions_spaarse

# создадим обучим модель LightFM

model_LightFM = LightFM( no_components=40, random_state=15, loss='logistic')
model_LightFM.fit(interactions_spaarse, epochs=20)

In [ ]:
n_items = interactions.shape[1]
np.arange(n_items)


array([   0,    1,    2, ..., 9716, 9717, 9718])

In [ ]:
genres_movie = [change_string (x) for x  in movies['genres'].values]

genres_movie[:5]

['Adventure Animation Children Comedy Fantasy',
 'Adventure Children Fantasy',
 'Comedy Romance',
 'Comedy Drama Romance',
 'Comedy']

In [ ]:
# Tfidf преобразование жанров в вектор

tfidf_2 = TfidfVectorizer()
x_tr = tfidf_2.fit_transform(genres_movie)

feat = tfidf_2.get_feature_names_out()
tfidf_movi = pd.DataFrame( x_tr.toarray(), columns = feat)

tfidf_movi['movieId'] = movies['movieId']

tfidf_movi

,action,adventure,animation,children,comedy,crime,documentary,drama,fantasy,filmnoir,...,imax,musical,mystery,nogenreslisted,romance,scifi,thriller,war,western,movieId
0,0.000000,0.416846,0.516225,0.504845,0.267586,0.0,0.0,0.000000,0.482990,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,1
1,0.000000,0.512361,0.000000,0.620525,0.000000,0.0,0.0,0.000000,0.593662,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,2
2,0.000000,0.000000,0.000000,0.000000,0.570915,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.821009,0.0,0.0,0.0,0.0,3
3,0.000000,0.000000,0.000000,0.000000,0.505015,0.0,0.0,0.466405,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.726241,0.0,0.0,0.0,0.0,4
4,0.000000,0.000000,0.000000,0.000000,1.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9737,0.436010,0.000000,0.614603,0.000000,0.318581,0.0,0.0,0.000000,0.575034,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,193581
9738,0.000000,0.000000,0.682937,0.000000,0.354002,0.0,0.0,0.000000,0.638968,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,193583
9739,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,1.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,193585
9740,0.578606,0.000000,0.815607,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,193587


In [ ]:
# объект Датасета для модели LightFM

ds = Dataset()

ds.fit(users = movies_ratings['userId'].unique(),
       items =  movies['movieId'],
       item_features = feat)

ds

In [ ]:
#  словарь взаимодействия -> объект (фильм): признак(жанр) -рейтинг

def transform_feat (feat, id_name):
  transform_feat = []

  for row in feat.to_dict(orient="records"):
    id_val = row[id_name]
    del row[id_name]
    feat_name = {key: value for key, value in row.items() if value !=0}
    transform_feat.append(( id_val, feat_name) )

  return transform_feat

mt = transform_feat( tfidf_movi, "movieId")
mt[:5]

[(1,
  {'adventure': 0.41684567364693936,
   'animation': 0.5162254711770092,
   'children': 0.5048454681396087,
   'comedy': 0.26758647689140014,
   'fantasy': 0.482990142708577}),
 (2,
  {'adventure': 0.5123612074824268,
   'children': 0.620525172745643,
   'fantasy': 0.5936619434123594}),
 (3, {'comedy': 0.5709154064399099, 'romance': 0.8210088907493954}),
 (4,
  {'comedy': 0.5050154397005037,
   'drama': 0.46640480307738325,
   'romance': 0.726240982959826}),
 (5, {'comedy': 1.0})]

In [ ]:
# формируем признаки для модели LightFM

item_feat_matrix = ds.build_item_features( mt)
item_feat_matrix

<9742x9762 sparse matrix of type '<class 'numpy.float32'>'
	with 31826 stored elements in Compressed Sparse Row format>

In [ ]:
dt = []
for i, row in movies_ratings.iterrows():
  dt.append( (row['userId'], row['movieId']) )

dt[:7]

[(1, 1), (5, 1), (7, 1), (15, 1), (17, 1), (18, 1), (19, 1)]

In [ ]:
interactions_matrix = ds.build_interactions(data = dt)[0]
interactions_matrix

<610x9742 sparse matrix of type '<class 'numpy.int32'>'
	with 100836 stored elements in COOrdinate format>

In [ ]:
model_LightFM_2 = LightFM(no_components=30, random_state=15, loss='bpr')
model_LightFM_2.fit(interactions_matrix, epochs=15, item_features = item_feat_matrix)

In [ ]:
n_items = interactions_matrix.shape[1]
user_id = 10
scor = model_LightFM_2.predict(user_id, np.arange(n_items))
# scor

scor = pd.Series(scor)
scor.index = movies['title']
scor.sort_values(ascending=False)

,0
title,
Apollo 13 (1995),4.473083
"Lion King, The (1994)",0.999660
Beauty and the Beast (1991),0.812634
Toy Story (1995),0.791159
Get Shorty (1995),0.784192
...,...
The Imitation Game (2014),-6.760725
Iron Man (2008),-6.930742
Inglourious Basterds (2009),-6.952345
